In [8]:
# all modules/dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

In [10]:
data_dir = "data_transfer"

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

#data transformation
data_transforms = {
   'train': transforms.Compose([
       transforms.Resize(size=(224,224)),
       transforms.ToTensor(),
       transforms.Normalize(mean, std)
   ]),
   'validation': transforms.Compose([
       transforms.Resize(size=(224,224)),
       transforms.ToTensor(),
       transforms.Normalize(mean, std)
   ]),
}





In [11]:
image_datasets = {
   x: datasets.ImageFolder(
       os.path.join(data_dir, x),
       transform=data_transforms[x]
   )
   for x in ['train', 'validation']
}

dataloaders = {
   x: torch.utils.data.DataLoader(
       image_datasets[x], batch_size=32,
       shuffle=True, num_workers=4
   )
   for x in ['train', 'validation']
}

In [12]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}

print(dataset_sizes)
class_names = image_datasets['train'].classes



{'train': 814, 'validation': 256}


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
vgg_based = torchvision.models.vgg19(pretrained=True)

## freeze the layers
for param in vgg_based.parameters():
    param.requires_grad = False

# Modify the last layer
number_features = vgg_based.classifier[6].in_features
features = list(vgg_based.classifier.children())[:-1] # Remove last layer
features.extend([torch.nn.Linear(number_features, len(class_names))])
vgg_based.classifier = torch.nn.Sequential(*features)

vgg_based = vgg_based.to(device)

print(vgg_based)

criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg_based.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\gagan/.cache\torch\checkpoints\vgg19-dcbb9e9d.pth
100%|███████████████████████████████████████████████████████████████| 574673361/574673361 [05:48<00:00, 1648299.63it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [15]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

       #set model to trainable
       # model.train()

        train_loss = 0

       # Iterate over data.
        for i, data in enumerate(dataloaders['train']):
            inputs , labels = data
            inputs = inputs.to(device)
            labels = labels.to(device) 

            optimizer.zero_grad()
          
            with torch.set_grad_enabled(True):
                outputs  = model(inputs)
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            print('{} Loss: {:.4f}'.format(
                'train', train_loss / dataset_sizes['train']))
          
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return model



In [16]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['validation']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
 
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {} truth: {}'.format(class_names[preds[j]], class_names[labels[j]]))
                img = inputs.cpu().data[j].numpy().transpose((1, 2, 0))
                img = std * img + mean
                ax.imshow(img)
 
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [17]:
vgg_based = train_model(vgg_based, criterion, optimizer_ft, num_epochs=25)



Epoch 0/24
----------
train Loss: 0.0292
train Loss: 0.0595
train Loss: 0.0856
train Loss: 0.1125
train Loss: 0.1394
train Loss: 0.1665
train Loss: 0.1936
train Loss: 0.2224
train Loss: 0.2507
train Loss: 0.2779
train Loss: 0.3049
train Loss: 0.3322
train Loss: 0.3578
train Loss: 0.3843
train Loss: 0.4087
train Loss: 0.4345
train Loss: 0.4653
train Loss: 0.4903
train Loss: 0.5160
train Loss: 0.5372
train Loss: 0.5672
train Loss: 0.5929
train Loss: 0.6183
train Loss: 0.6430
train Loss: 0.6680
train Loss: 0.6783
Epoch 1/24
----------
train Loss: 0.0220
train Loss: 0.0455
train Loss: 0.0676
train Loss: 0.0956
train Loss: 0.1186
train Loss: 0.1428
train Loss: 0.1652
train Loss: 0.1937
train Loss: 0.2200
train Loss: 0.2407
train Loss: 0.2635
train Loss: 0.2922
train Loss: 0.3203
train Loss: 0.3413
train Loss: 0.3659
train Loss: 0.3908
train Loss: 0.4091
train Loss: 0.4327
train Loss: 0.4576
train Loss: 0.4797
train Loss: 0.5028
train Loss: 0.5273
train Loss: 0.5496
train Loss: 0.5675
train 

KeyboardInterrupt: 

In [ ]:
visualize_model(vgg_based)

plt.show()